In [2]:
import tensorflow as tf
hello=tf.constant('Hello,TensorFlow!')
sess=tf.Session()
print(sess.run(hello))

b'Hello,TensorFlow!'


In [3]:
import random
# import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV,cross_val_score
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.linear_model import LogisticRegression
import pickle

import random
import numpy as np

from sklearn import svm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn import preprocessing


In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV,cross_val_score
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.tree import DecisionTreeClassifier, export_graphviz


import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

from sklearn.model_selection import cross_val_score, StratifiedKFold, GridSearchCV, train_test_split, TimeSeriesSplit
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score , roc_curve, auc, log_loss
import lightgbm as lgb
from lightgbm.sklearn import LGBMClassifier
import warnings
warnings.filterwarnings('ignore')

In [5]:
import pandas as pd

import numpy as np

import sys,os,pickle

# from sklearn.cross_validation import StratifiedShuffleSplit

from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.decomposition import TruncatedSVD

from scipy.spatial import distance

from scipy.stats import pearsonr

import math

import networkx as nx

import lightgbm as lgb

from collections import Counter

from sklearn.metrics import log_loss,accuracy_score,f1_score

In [6]:
def cosine(a,b):
    res=0
    if np.linalg.norm(a)!=0 and np.linalg.norm(b)!=0:
        res=distance.cosine(a,b)
    return res

In [7]:
submission_df = pd.read_csv('sample_submission_only_headers.csv')
submission_df.head()

,id,is_chat


In [8]:
train_df = pd.read_csv('train.csv')
train_df.head()

,node1_id,node2_id,is_chat
0,8446602,6636127,0
1,1430102,7433949,0
2,2803017,8372333,0
3,4529348,894645,0
4,5096572,4211638,0


In [9]:
train_df['is_chat'].value_counts(normalize = True)

0    0.968133
1    0.031867
Name: is_chat, dtype: float64

In [10]:
test_df = pd.read_csv('test.csv')
test_df.head()

,id,node1_id,node2_id
0,1,7107094,8010772
1,2,7995251,2805801
2,3,2804693,8059549
3,4,4812472,7332370
4,5,5009985,4511909


In [11]:
user_features_df = pd.read_csv('user_features.csv')
user_features_df.head()

,node_id,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13
0,2,14,14,14,12,12,12,7,7,7,0,0,0,15
1,3,31,9,7,31,16,12,31,15,12,31,15,12,8
2,4,0,0,0,0,0,0,0,0,0,0,0,0,7
3,5,31,4,1,31,7,1,31,9,1,31,9,0,15
4,6,31,27,20,31,24,14,31,20,10,31,20,5,7


In [12]:
user_features_df.columns

Index(['node_id', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10',
       'f11', 'f12', 'f13'],
      dtype='object')

In [13]:
# read python dict back from the file
pkl_file = open('user_features_df_dict.pkl', 'rb')
user_features_df_dict = pickle.load(pkl_file)
pkl_file.close()



In [14]:
pkl_file_features_df = pd.DataFrame(user_features_df_dict.items(), columns = ['node_id', 'list_vals']) 

In [15]:
pkl_file_features_df.head(5)

,node_id,list_vals
0,2,"[14, 14, 14, 12, 12, 12, 7, 7, 7, 0, 0, 0, 15]"
1,3,"[31, 9, 7, 31, 16, 12, 31, 15, 12, 31, 15, 12, 8]"
2,4,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7]"
3,5,"[31, 4, 1, 31, 7, 1, 31, 9, 1, 31, 9, 0, 15]"
4,6,"[31, 27, 20, 31, 24, 14, 31, 20, 10, 31, 20, 5..."


In [16]:
features={}

## Train Data Preparation

In [17]:
#Train
training_set_reduced = train_df.sample(frac=0.0005)
training_set_reduced.head(3)

,node1_id,node2_id,is_chat
42095428,6870043,5414830,0
45389108,8145709,5665936,1
54219129,4678200,4480021,0


In [18]:
print(training_set_reduced.shape)
print(train_df.head(3))

(35331, 3)
   node1_id  node2_id  is_chat
0   8446602   6636127        0
1   1430102   7433949        0
2   2803017   8372333        0


In [30]:
s0_train = pd.merge(training_set_reduced, pkl_file_features_df, how='left', left_on='node1_id', right_on='node_id')
s0_train.drop(['node_id'],axis=1,inplace= True)
s0_train.rename(columns={'list_vals': 'node1_list_vals' }, inplace=True)
print(s0_train.shape)
print(s0_train.head(4))

s00_train = pd.merge(s0_train, pkl_file_features_df, how='left', left_on='node2_id', right_on='node_id')
s00_train.drop(['node_id'],axis=1,inplace= True)
s00_train.rename(columns={'list_vals': 'node2_list_vals' }, inplace=True)
print(s00_train.shape)
print(s00_train.head(4))

(35331, 4)
   node1_id  node2_id  is_chat  \
0   6870043   5414830        0   
1   8145709   5665936        1   
2   4678200   4480021        0   
3   3733510    855841        0   

                                     node1_list_vals  
0    [31, 12, 0, 31, 12, 0, 31, 11, 0, 31, 12, 0, 4]  
1  [30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 3...  
2            [0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 8]  
3        [27, 4, 0, 25, 6, 0, 22, 5, 0, 19, 3, 0, 7]  
(35331, 5)
   node1_id  node2_id  is_chat  \
0   6870043   5414830        0   
1   8145709   5665936        1   
2   4678200   4480021        0   
3   3733510    855841        0   

                                     node1_list_vals  \
0    [31, 12, 0, 31, 12, 0, 31, 11, 0, 31, 12, 0, 4]   
1  [30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 3...   
2            [0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 8]   
3        [27, 4, 0, 25, 6, 0, 22, 5, 0, 19, 3, 0, 7]   

                                   node2_list_vals  
0          [0, 0, 0, 0, 0, 

In [31]:
s1_train = pd.merge(s00_train, user_features_df, how='left', left_on='node1_id', right_on='node_id')
print(s1_train.shape)
print(s1_train.head(4))

s1_train.drop(['node_id'],axis=1,inplace= True)
s1_train.rename(columns={'f1': 'f1_node1', 'f2': 'f2_node1', 'f3': 'f3_node1',
                   'f4': 'f4_node1', 'f5': 'f5_node1', 'f6': 'f6_node1',
                   'f7': 'f7_node1', 'f8': 'f8_node1', 'f9': 'f9_node1',
                   'f10': 'f10_node1', 'f11': 'f11_node1', 'f12': 'f12_node1',
                   'f13': 'f13_node1'                  
                  }, inplace=True)

print(s1_train.shape)
print(s1_train.head(4))


(35331, 19)
   node1_id  node2_id  is_chat  \
0   6870043   5414830        0   
1   8145709   5665936        1   
2   4678200   4480021        0   
3   3733510    855841        0   

                                     node1_list_vals  \
0    [31, 12, 0, 31, 12, 0, 31, 11, 0, 31, 12, 0, 4]   
1  [30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 3...   
2            [0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 8]   
3        [27, 4, 0, 25, 6, 0, 22, 5, 0, 19, 3, 0, 7]   

                                   node2_list_vals  node_id  f1  f2  f3  f4  \
0          [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 2]  6870043  31  12   0  31   
1        [0, 0, 0, 1, 0, 0, 7, 0, 0, 14, 0, 0, 15]  8145709  30  30  30  30   
2  [31, 2, 0, 31, 7, 3, 28, 10, 6, 26, 14, 10, 15]  4678200   0   0   0   2   
3     [31, 2, 0, 31, 0, 0, 29, 1, 1, 22, 1, 1, 15]  3733510  27   4   0  25   

   f5  f6  f7  f8  f9  f10  f11  f12  f13  
0  12   0  31  11   0   31   12    0    4  
1  30  30  30  30  30   30   30   30    7  
2   2   

In [32]:
s2_train = pd.merge(s1_train, user_features_df, how='left', left_on='node2_id', right_on='node_id')
print(s2_train.shape)
print(s2_train.head(4))

(35331, 32)
   node1_id  node2_id  is_chat  \
0   6870043   5414830        0   
1   8145709   5665936        1   
2   4678200   4480021        0   
3   3733510    855841        0   

                                     node1_list_vals  \
0    [31, 12, 0, 31, 12, 0, 31, 11, 0, 31, 12, 0, 4]   
1  [30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 3...   
2            [0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 8]   
3        [27, 4, 0, 25, 6, 0, 22, 5, 0, 19, 3, 0, 7]   

                                   node2_list_vals  f1_node1  f2_node1  \
0          [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 2]        31        12   
1        [0, 0, 0, 1, 0, 0, 7, 0, 0, 14, 0, 0, 15]        30        30   
2  [31, 2, 0, 31, 7, 3, 28, 10, 6, 26, 14, 10, 15]         0         0   
3     [31, 2, 0, 31, 0, 0, 29, 1, 1, 22, 1, 1, 15]        27         4   

   f3_node1  f4_node1  f5_node1  ...  f4  f5  f6  f7  f8  f9  f10  f11  f12  \
0         0        31        12  ...   0   0   0   0   0   0    1    1    0   
1      

In [33]:
s2_train.columns

Index(['node1_id', 'node2_id', 'is_chat', 'node1_list_vals', 'node2_list_vals',
       'f1_node1', 'f2_node1', 'f3_node1', 'f4_node1', 'f5_node1', 'f6_node1',
       'f7_node1', 'f8_node1', 'f9_node1', 'f10_node1', 'f11_node1',
       'f12_node1', 'f13_node1', 'node_id', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6',
       'f7', 'f8', 'f9', 'f10', 'f11', 'f12', 'f13'],
      dtype='object')

In [34]:
s2_train["g_jaccard_index"] = ""
s2_train["g_neighbour_sqrt"] = ""
s2_train["g_neighbour_cosine"] = ""
s2_train["g_neighbour_pearson"] = ""

In [44]:
del [[s1_train]]
import gc
gc.collect()
s1_train=pd.DataFrame()

s2_train['f1_sum'] = s2_train['f1_node1']+s2_train['f1']
s2_train['f2_sum'] = s2_train['f2_node1']+s2_train['f2']
s2_train['f3_sum'] = s2_train['f3_node1']+s2_train['f3']
s2_train['f4_sum'] = s2_train['f4_node1']+s2_train['f4']
s2_train['f5_sum'] = s2_train['f5_node1']+s2_train['f5']
s2_train['f6_sum'] = s2_train['f6_node1']+s2_train['f6']
s2_train['f7_sum'] = s2_train['f7_node1']+s2_train['f7']
s2_train['f8_sum'] = s2_train['f8_node1']+s2_train['f8']
s2_train['f9_sum'] = s2_train['f9_node1']+s2_train['f9']
s2_train['f10_sum'] = s2_train['f10_node1']+s2_train['f10']
s2_train['f11_sum'] = s2_train['f11_node1']+s2_train['f11']
s2_train['f12_sum'] = s2_train['f12_node1']+s2_train['f12']
s2_train['f13_sum'] = s2_train['f13_node1']+s2_train['f13']

# sn=G.neighbors(row.node1_id)
# tn=G.neighbors(row.node2_id)

for i in range(s2_train.shape[0]):

    common_ns=len(set(list(s2_train['node1_list_vals'][i])).intersection(set(list(s2_train['node2_list_vals'][i]))))

    s2_train['g_jaccard_index'][i] = common_ns/((1e-6+len(set((s2_train['node1_list_vals'][i]+s2_train['node2_list_vals'][i])))))

   
    sn_vec,tn_vec=get_vectors(s2_train['node1_list_vals'][i],s2_train['node2_list_vals'][i])

    s2_train['g_neighbour_sqrt'][i] = common_ns/(math.sqrt(len(list(s2_train['node1_list_vals'][i]))+
                                                        len(list(s2_train['node2_list_vals'][i])))+1e-6)

    s2_train['g_neighbour_cosine'][i] = distance.cosine(sn_vec,tn_vec)

    s2_train['g_neighbour_pearson'][i] = pearsonr(sn_vec,tn_vec)[0]
        

    

s2_train.drop([
    'node1_id', 'node2_id', 
    'f1_node1', 'f2_node1', 'f3_node1',
       'f4_node1', 'f5_node1', 'f6_node1', 'f7_node1', 'f8_node1', 'f9_node1',
       'f10_node1', 'f11_node1', 'f12_node1', 'f13_node1', 'node_id', 'f1',
       'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10', 'f11', 'f12',
       'f13', 'node1_list_vals', 'node2_list_vals'], axis=1, inplace=True)


print(s2_train.shape)
print(s2_train.head(3))

In [118]:
s2_train.shape

(353309, 49)

In [27]:
def get_vectors(a_list,b_list):


    id2index=dict([(id,i) for i,id in enumerate(set((a_list+b_list)))])

    a=np.zeros((len(id2index),))

    b=np.zeros((len(id2index),))

    for key in a_list:

        a[id2index[key]]=1

    for key in b_list:

        b[id2index[key]]=1

    return a,b

## Test Data Preparation

In [ ]:
s0 = pd.merge(test_df, pkl_file_features_df, how='left', left_on='node1_id', right_on='node_id')
s0.drop(['node_id'],axis=1,inplace= True)
s0.rename(columns={'list_vals': 'node1_list_vals' }, inplace=True)
print(s0.shape)
print(s0.head(4))

s00 = pd.merge(s0, pkl_file_features_df, how='left', left_on='node2_id', right_on='node_id')
s00.drop(['node_id'],axis=1,inplace= True)
s00.rename(columns={'list_vals': 'node2_list_vals' }, inplace=True)
print(s00.shape)
print(s00.head(4))

In [ ]:
s1 = pd.merge(s00, user_features_df, how='left', left_on='node1_id', right_on='node_id')
print(s1.shape)
print(s1.head(4))

s1.drop(['node_id'],axis=1,inplace= True)
s1.rename(columns={'f1': 'f1_node1', 'f2': 'f2_node1', 'f3': 'f3_node1',
                   'f4': 'f4_node1', 'f5': 'f5_node1', 'f6': 'f6_node1',
                   'f7': 'f7_node1', 'f8': 'f8_node1', 'f9': 'f9_node1',
                   'f10': 'f10_node1', 'f11': 'f11_node1', 'f12': 'f12_node1',
                   'f13': 'f13_node1'                  
                  }, inplace=True)

print(s1.shape)
print(s1.head(4))

In [ ]:
s2 = pd.merge(s1, user_features_df, how='left', left_on='node2_id', right_on='node_id')
print(s2.shape)
print(s2.head(4))

In [ ]:
s2.columns

In [ ]:
del [[s1]]
import gc
gc.collect()
s1=pd.DataFrame()

In [ ]:
s2["g_jaccard_index"] = ""
s2["g_neighbour_sqrt"] = ""
s2["g_neighbour_cosine"] = ""
s2["g_neighbour_pearson"] = ""


In [ ]:
s2['f1_sum'] = s2['f1_node1']+s2['f1']
s2['f2_sum'] = s2['f2_node1']+s2['f2']
s2['f3_sum'] = s2['f3_node1']+s2['f3']
s2['f4_sum'] = s2['f4_node1']+s2['f4']
s2['f5_sum'] = s2['f5_node1']+s2['f5']
s2['f6_sum'] = s2['f6_node1']+s2['f6']
s2['f7_sum'] = s2['f7_node1']+s2['f7']
s2['f8_sum'] = s2['f8_node1']+s2['f8']
s2['f9_sum'] = s2['f9_node1']+s2['f9']
s2['f10_sum'] = s2['f10_node1']+s2['f10']
s2['f11_sum'] = s2['f11_node1']+s2['f11']
s2['f12_sum'] = s2['f12_node1']+s2['f12']
s2['f13_sum'] = s2['f13_node1']+s2['f13']


for i in range(s2.shape[0]):

    common_ns=len(set(list(s2['node1_list_vals'][i])).intersection(set(list(s2['node2_list_vals'][i]))))

    s2['g_jaccard_index'][i] = common_ns/((1e-6+len(set((s2['node1_list_vals'][i]+s2['node2_list_vals'][i])))))

    
    sn_vec,tn_vec=get_vectors(s2['node1_list_vals'][i],s2['node2_list_vals'][i])

    s2['g_neighbour_sqrt'][i] = common_ns/(math.sqrt(len(list(s2['node1_list_vals'][i]))+
                                                        len(list(s2['node2_list_vals'][i])))+1e-6)

    s2['g_neighbour_cosine'][i] = distance.cosine(sn_vec,tn_vec)

    s2['g_neighbour_pearson'][i] = pearsonr(sn_vec,tn_vec)[0]
    
    

s2.drop([
    'node1_id', 'node2_id',
         'f1_node1', 'f2_node1', 'f3_node1',
       'f4_node1', 'f5_node1', 'f6_node1', 'f7_node1', 'f8_node1', 'f9_node1',
       'f10_node1', 'f11_node1', 'f12_node1', 'f13_node1', 'node_id', 'f1',
       'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10', 'f11', 'f12',
       'f13', 'node1_list_vals', 'node2_list_vals'], axis=1, inplace=True)


print(s2.shape)
print(s2.head(3))


In [30]:
training_features = s2_train.values

In [31]:
s2_train.head(2)

,is_chat,f1_sum,f2_sum,f3_sum,f4_sum,f5_sum,f6_sum,f7_sum,f8_sum,f9_sum,f10_sum,f11_sum,f12_sum,f13_sum
0,0,40,30,21,47,32,21,53,35,19,57,41,20,17
1,0,31,31,29,31,31,29,31,31,29,31,30,28,22


In [32]:
training_features = s2_train.drop(['is_chat'], axis=1)
training_features.head(3)

,f1_sum,f2_sum,f3_sum,f4_sum,f5_sum,f6_sum,f7_sum,f8_sum,f9_sum,f10_sum,f11_sum,f12_sum,f13_sum
0,40,30,21,47,32,21,53,35,19,57,41,20,17
1,31,31,29,31,31,29,31,31,29,31,30,28,22
2,23,22,3,16,16,3,11,11,2,4,4,1,30


In [33]:
training_features = training_features.values

In [34]:
training_features

array([[40, 30, 21, ..., 41, 20, 17],
       [31, 31, 29, ..., 30, 28, 22],
       [23, 22,  3, ...,  4,  1, 30],
       ...,
       [59, 21, 10, ..., 23, 10, 20],
       [31, 19, 10, ..., 26, 17, 18],
       [62, 55, 36, ..., 54, 34, 22]], dtype=int64)

In [35]:
# scale
training_features = preprocessing.scale(training_features)


labels = list(s2_train['is_chat'])

labels_array = np.array(labels)
# labels_array.to_csv('labels_array.csv',index= False)


X_train, X_valid, y_train, y_valid = \
    train_test_split(training_features, labels_array, 
                     test_size=0.3, random_state=17)

logit_pipe = Pipeline([
                       ('logit', LogisticRegression(C=1, random_state=17, solver='liblinear'))])
logit_pipe.fit(X_train, y_train)
logit_valid_pred = logit_pipe.predict_proba(X_valid)[:, 1]

roc_auc_score(y_valid, logit_valid_pred)

# 0.7514437922008632

0.7517654986153098

In [36]:
s2.head(2)

,id,f1_sum,f2_sum,f3_sum,f4_sum,f5_sum,f6_sum,f7_sum,f8_sum,f9_sum,f10_sum,f11_sum,f12_sum,f13_sum
0,1,62,30,3,62,30,4,62,33,4,62,35,5,30
1,2,0,0,0,0,0,0,0,0,0,6,0,0,14


In [37]:
testing_features = s2.drop(['id'],axis=1).values
# scale
testing_features = preprocessing.scale(testing_features)

# issue predictions
# predictions_logistic = list(classifier.predict_proba(testing_features)[:,1])
# predictions_logistic = list(logit_grid_searcher.predict_proba(testing_features)[:,1])



In [ ]:
submission_df['id'] = test_df['id']
submission_df['is_chat'] = pd.DataFrame(predictions_logistic)


submission_df.to_csv('submission_df_lg_baseline_v3.csv',index= False)

In [45]:
# Split on 5 folds
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=17)



logit_pipe = Pipeline([
                       ('logit', LogisticRegression(C=1, random_state=17, solver='liblinear'))])

c_values = np.logspace(-1, 1, 20)

paramgrid = {'logit__C' : c_values  }

logit_grid_searcher = GridSearchCV(estimator=logit_pipe, param_grid=paramgrid,
                                  scoring='roc_auc', n_jobs=1, cv=skf, verbose=1)

logit_grid_searcher.fit(X_train,y_train)
y_prd=logit_grid_searcher.predict_proba(X_valid)[:, 1]
roc_auc_score(y_valid,y_prd)

Fitting 5 folds for each of 20 candidates, totalling 100 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:  2.0min finished


0.7520414212033201

In [44]:
logit_grid_searcher.best_params_, logit_grid_searcher.best_score_

({'logit__C': 0.1}, 0.7483230756118662)

In [46]:
logit_pipe = Pipeline([
                       ('logit', LogisticRegression(C=0.1, random_state=17,
                                                    solver='liblinear'))])

logit_pipe.fit(X_train, y_train)
logit_valid_pred = logit_pipe.predict_proba(X_valid)[:, 1]

print(roc_auc_score(y_valid, logit_valid_pred))


logit_pipe.fit(training_features, labels_array)
logit_test_pred = logit_pipe.predict_proba(testing_features)[:, 1]

submission_df['id'] = test_df['id']
submission_df['is_chat'] = pd.DataFrame(logit_test_pred)


submission_df.to_csv('submission_df_logit_5fold.csv',index= False)

0.7520414212033201


In [38]:
## LightGBM
train_data = lgb.Dataset(X_train, label=y_train)
valid_data = lgb.Dataset(X_valid, label=y_valid)

param = {'num_leaves':200, 'num_trees':100 ,'objective':'binary'}
num_round = 300
bst_lgb = lgb.train(param, train_data, num_round, valid_sets=[valid_data], early_stopping_rounds=20)

[1]	valid_0's binary_logloss: 0.133455
Training until validation scores don't improve for 20 rounds.
[2]	valid_0's binary_logloss: 0.129937
[3]	valid_0's binary_logloss: 0.127501
[4]	valid_0's binary_logloss: 0.125672
[5]	valid_0's binary_logloss: 0.124199
[6]	valid_0's binary_logloss: 0.123022
[7]	valid_0's binary_logloss: 0.122031
[8]	valid_0's binary_logloss: 0.121224
[9]	valid_0's binary_logloss: 0.120514
[10]	valid_0's binary_logloss: 0.119955
[11]	valid_0's binary_logloss: 0.119469
[12]	valid_0's binary_logloss: 0.119046
[13]	valid_0's binary_logloss: 0.118681
[14]	valid_0's binary_logloss: 0.118333
[15]	valid_0's binary_logloss: 0.118068
[16]	valid_0's binary_logloss: 0.117814
[17]	valid_0's binary_logloss: 0.117606
[18]	valid_0's binary_logloss: 0.117411
[19]	valid_0's binary_logloss: 0.117257
[20]	valid_0's binary_logloss: 0.117131
[21]	valid_0's binary_logloss: 0.117003
[22]	valid_0's binary_logloss: 0.116903
[23]	valid_0's binary_logloss: 0.116796
[24]	valid_0's binary_loglo

In [39]:
lgb_pred = (bst_lgb.predict(X_valid.astype(np.float32), num_iteration=bst_lgb.best_iteration))

In [40]:
roc_auc_score(y_valid, lgb_pred)

0.8174645949974516

In [44]:
lgb_pred_test= (bst_lgb.predict(testing_features.astype(np.float32), 
                                    num_iteration=bst_lgb.best_iteration))

In [45]:
lgb_pred_test

array([0.02818028, 0.00406865, 0.00615933, ..., 0.01332921, 0.02218711,
       0.00801583])

In [46]:
submission_df['id'] = test_df['id']
submission_df['is_chat'] = pd.DataFrame(lgb_pred_test)


submission_df.to_csv('submission_df_lgbm_baseline.csv',index= False)

In [47]:
params = {'boosting_type': 'gbdt',
          'max_depth' : -1,
          'objective': 'binary', 
          'nthread': 5, # Updated from nthread
          'num_leaves': 64, 
          'learning_rate': 0.05, 
          'max_bin': 512, 
          'subsample_for_bin': 200,
          'subsample': 1, 
          'subsample_freq': 1, 
          'colsample_bytree': 0.8, 
          'reg_alpha': 5, 
          'reg_lambda': 10,
          'min_split_gain': 0.5, 
          'min_child_weight': 1, 
          'min_child_samples': 5, 
          'scale_pos_weight': 1,
          'num_class' : 1,
          'metric' : 'auc'}


In [48]:
# Create parameters to search
gridParams = {
    'learning_rate': [0.005],
    'n_estimators': [8,16,24],
    'num_leaves': [6,8,12,16],
    'boosting_type' : ['gbdt'],
    'objective' : ['binary'],
    'random_state' : [501], # Updated from 'seed'
    'colsample_bytree' : [0.64, 0.65, 0.66, 0.67],
    'subsample' : [0.7,0.75],
    'reg_alpha' : [1,1.2, 1.4, 1.6, 1.8],
    'reg_lambda' : [1,1.2,1.4],
    }

In [49]:
import lightgbm as lgb
mdl = lgb.LGBMClassifier(boosting_type= 'gbdt', 
          objective = 'binary', 
          n_jobs = 5, # Updated from 'nthread' 
          silent = True,
          max_depth = params['max_depth'],
          max_bin = params['max_bin'], 
          subsample_for_bin = params['subsample_for_bin'],
          subsample = params['subsample'], 
          subsample_freq = params['subsample_freq'], 
          min_split_gain = params['min_split_gain'], 
          min_child_weight = params['min_child_weight'], 
          min_child_samples = params['min_child_samples'], 
          scale_pos_weight = params['scale_pos_weight'])


In [50]:
# To view the default model params:
mdl.get_params().keys()

# Create the grid
grid = GridSearchCV(mdl, gridParams, verbose=0, cv=4, n_jobs=2)
# Run the grid
grid.fit(X_train, y_train)

# Print the best parameters found
print(grid.best_params_)
print(grid.best_score_)

{'boosting_type': 'gbdt', 'colsample_bytree': 0.64, 'learning_rate': 0.005, 'n_estimators': 8, 'num_leaves': 6, 'objective': 'binary', 'random_state': 501, 'reg_alpha': 1, 'reg_lambda': 1, 'subsample': 0.7}
0.9681096249332838


In [51]:
# Using parameters already set above, replace in the best from the grid search
params['colsample_bytree'] = grid.best_params_['colsample_bytree']
params['learning_rate'] = grid.best_params_['learning_rate'] 
# params['max_bin'] = grid.best_params_['max_bin']
params['num_leaves'] = grid.best_params_['num_leaves']
params['reg_alpha'] = grid.best_params_['reg_alpha']
params['reg_lambda'] = grid.best_params_['reg_lambda']
params['subsample'] = grid.best_params_['subsample']
# params['subsample_for_bin'] = grid.best_params_['subsample_for_bin']

print('Fitting with params: ')
print(params)


Fitting with params: 
{'boosting_type': 'gbdt', 'max_depth': -1, 'objective': 'binary', 'nthread': 5, 'num_leaves': 6, 'learning_rate': 0.005, 'max_bin': 512, 'subsample_for_bin': 200, 'subsample': 0.7, 'subsample_freq': 1, 'colsample_bytree': 0.64, 'reg_alpha': 1, 'reg_lambda': 1, 'min_split_gain': 0.5, 'min_child_weight': 1, 'min_child_samples': 5, 'scale_pos_weight': 1, 'num_class': 1, 'metric': 'auc'}


In [ ]:
# Fitting with params: 
# {'boosting_type': 'gbdt', 'max_depth': -1, 'objective': 'binary', 'nthread': 5, 'num_leaves': 6, 'learning_rate': 0.005, 'max_bin': 512, 'subsample_for_bin': 200, 'subsample': 0.7, 'subsample_freq': 1, 'colsample_bytree': 0.64, 'reg_alpha': 1, 'reg_lambda': 1, 'min_split_gain': 
#  0.5, 'min_child_weight': 1, 'min_child_samples': 5, 'scale_pos_weight': 1, 
#  'num_class': 1, 'metric': 'auc'}

In [53]:
## LightGBM
train_data = lgb.Dataset(X_train, label=y_train)
valid_data = lgb.Dataset(X_valid, label=y_valid)

# param = {'num_leaves':200, 'num_trees':100 ,'objective':'binary'}
num_round = 300
bst_lgb = lgb.train(params, train_data, num_round, valid_sets=[valid_data], early_stopping_rounds=100)

[1]	valid_0's auc: 0.742608
Training until validation scores don't improve for 100 rounds.
[2]	valid_0's auc: 0.752295
[3]	valid_0's auc: 0.753191
[4]	valid_0's auc: 0.753713
[5]	valid_0's auc: 0.753566
[6]	valid_0's auc: 0.754563
[7]	valid_0's auc: 0.757298
[8]	valid_0's auc: 0.75724
[9]	valid_0's auc: 0.757422
[10]	valid_0's auc: 0.757437
[11]	valid_0's auc: 0.758124
[12]	valid_0's auc: 0.758054
[13]	valid_0's auc: 0.758438
[14]	valid_0's auc: 0.758078
[15]	valid_0's auc: 0.75802
[16]	valid_0's auc: 0.758106
[17]	valid_0's auc: 0.758469
[18]	valid_0's auc: 0.758796
[19]	valid_0's auc: 0.758796
[20]	valid_0's auc: 0.758918
[21]	valid_0's auc: 0.758865
[22]	valid_0's auc: 0.758928
[23]	valid_0's auc: 0.759935
[24]	valid_0's auc: 0.759973
[25]	valid_0's auc: 0.760324
[26]	valid_0's auc: 0.76024
[27]	valid_0's auc: 0.761666
[28]	valid_0's auc: 0.761753
[29]	valid_0's auc: 0.761921
[30]	valid_0's auc: 0.762064
[31]	valid_0's auc: 0.762001
[32]	valid_0's auc: 0.761969
[33]	valid_0's auc: 0

[280]	valid_0's auc: 0.784875
[281]	valid_0's auc: 0.78492
[282]	valid_0's auc: 0.785006
[283]	valid_0's auc: 0.785015
[284]	valid_0's auc: 0.785073
[285]	valid_0's auc: 0.785078
[286]	valid_0's auc: 0.785143
[287]	valid_0's auc: 0.785106
[288]	valid_0's auc: 0.785162
[289]	valid_0's auc: 0.785208
[290]	valid_0's auc: 0.785244
[291]	valid_0's auc: 0.785294
[292]	valid_0's auc: 0.785321
[293]	valid_0's auc: 0.785334
[294]	valid_0's auc: 0.785395
[295]	valid_0's auc: 0.785428
[296]	valid_0's auc: 0.785476
[297]	valid_0's auc: 0.785486
[298]	valid_0's auc: 0.785513
[299]	valid_0's auc: 0.785516
[300]	valid_0's auc: 0.785541
Did not meet early stopping. Best iteration is:
[300]	valid_0's auc: 0.785541


In [54]:
lgb_pred_test= (bst_lgb.predict(testing_features.astype(np.float32), 
                                    num_iteration=bst_lgb.best_iteration))

In [55]:
submission_df['id'] = test_df['id']
submission_df['is_chat'] = pd.DataFrame(lgb_pred_test)
submission_df.to_csv('submission_df_lgbm_tuned_baseline.csv',index= False)

### Catboost - Classifier

In [51]:
training_features_catboost = s2_train.drop(['is_chat'], axis=1)
training_features_catboost.head(3)

,f1_sum,f2_sum,f3_sum,f4_sum,f5_sum,f6_sum,f7_sum,f8_sum,f9_sum,f10_sum,f11_sum,f12_sum,f13_sum
0,0,0,0,0,0,0,0,0,0,7,2,0,30
1,31,31,31,32,31,31,34,33,31,34,33,31,22
2,61,38,31,60,38,31,60,41,35,60,45,37,22


In [55]:
X_train, X_valid, y_train, y_valid = \
    train_test_split(training_features_catboost, s2_train['is_chat'], 
                     test_size=0.3, random_state=17)

## Catboost


from catboost import Pool, CatBoostClassifier, cv


cate_features_index = [1,2,3,4,5,6,7,8,9,10,11,12]
#let us make the catboost model, use_best_model params will make the model prevent overfitting
model = CatBoostClassifier(eval_metric='AUC',use_best_model=True,random_seed=17)
#now just to make the model to fit the data
model.fit(X_train,y_train, cat_features=cate_features_index,eval_set=(X_valid,y_valid))

0:	test: 0.6231172	best: 0.6231172 (0)	total: 7.09s	remaining: 1h 58m
1:	test: 0.6235304	best: 0.6235304 (1)	total: 14.1s	remaining: 1h 57m 31s
2:	test: 0.6237925	best: 0.6237925 (2)	total: 20.6s	remaining: 1h 54m 1s
3:	test: 0.6237925	best: 0.6237925 (2)	total: 27s	remaining: 1h 52m 4s
4:	test: 0.6237925	best: 0.6237925 (2)	total: 34.1s	remaining: 1h 53m
5:	test: 0.6631236	best: 0.6631236 (5)	total: 40.7s	remaining: 1h 52m 29s
6:	test: 0.7333604	best: 0.7333604 (6)	total: 46.5s	remaining: 1h 50m 1s
7:	test: 0.7589924	best: 0.7589924 (7)	total: 52s	remaining: 1h 47m 27s
8:	test: 0.7693211	best: 0.7693211 (8)	total: 58.3s	remaining: 1h 46m 58s
9:	test: 0.7772492	best: 0.7772492 (9)	total: 1m 3s	remaining: 1h 44m 9s
10:	test: 0.7821499	best: 0.7821499 (10)	total: 1m 7s	remaining: 1h 40m 55s
11:	test: 0.7839898	best: 0.7839898 (11)	total: 1m 12s	remaining: 1h 38m 49s
12:	test: 0.7852527	best: 0.7852527 (12)	total: 1m 15s	remaining: 1h 36m 6s
13:	test: 0.7865333	best: 0.7865333 (13)	total:

108:	test: 0.8095665	best: 0.8095665 (108)	total: 11m 56s	remaining: 1h 37m 37s
109:	test: 0.8095677	best: 0.8095677 (109)	total: 12m 2s	remaining: 1h 37m 25s
110:	test: 0.8095727	best: 0.8095727 (110)	total: 12m 9s	remaining: 1h 37m 23s
111:	test: 0.8095802	best: 0.8095802 (111)	total: 12m 16s	remaining: 1h 37m 16s
112:	test: 0.8095796	best: 0.8095802 (111)	total: 12m 22s	remaining: 1h 37m 11s
113:	test: 0.8095901	best: 0.8095901 (113)	total: 12m 30s	remaining: 1h 37m 9s
114:	test: 0.8095887	best: 0.8095901 (113)	total: 12m 35s	remaining: 1h 36m 54s
115:	test: 0.8095907	best: 0.8095907 (115)	total: 12m 42s	remaining: 1h 36m 50s
116:	test: 0.8096959	best: 0.8096959 (116)	total: 12m 49s	remaining: 1h 36m 49s
117:	test: 0.8096949	best: 0.8096959 (116)	total: 12m 56s	remaining: 1h 36m 40s
118:	test: 0.8097000	best: 0.8097000 (118)	total: 13m 2s	remaining: 1h 36m 32s
119:	test: 0.8096982	best: 0.8097000 (118)	total: 13m 9s	remaining: 1h 36m 29s
120:	test: 0.8097015	best: 0.8097015 (120)	to

211:	test: 0.8113295	best: 0.8113295 (211)	total: 24m 3s	remaining: 1h 29m 25s
212:	test: 0.8113354	best: 0.8113354 (212)	total: 24m 11s	remaining: 1h 29m 23s
213:	test: 0.8113367	best: 0.8113367 (213)	total: 24m 17s	remaining: 1h 29m 14s
214:	test: 0.8113362	best: 0.8113367 (213)	total: 24m 24s	remaining: 1h 29m 6s
215:	test: 0.8113382	best: 0.8113382 (215)	total: 24m 31s	remaining: 1h 29m 1s
216:	test: 0.8113670	best: 0.8113670 (216)	total: 24m 37s	remaining: 1h 28m 51s
217:	test: 0.8113664	best: 0.8113670 (216)	total: 24m 44s	remaining: 1h 28m 45s
218:	test: 0.8113680	best: 0.8113680 (218)	total: 24m 51s	remaining: 1h 28m 38s
219:	test: 0.8113778	best: 0.8113778 (219)	total: 24m 58s	remaining: 1h 28m 31s
220:	test: 0.8113794	best: 0.8113794 (220)	total: 25m 6s	remaining: 1h 28m 30s
221:	test: 0.8113729	best: 0.8113794 (220)	total: 25m 13s	remaining: 1h 28m 23s
222:	test: 0.8113873	best: 0.8113873 (222)	total: 25m 20s	remaining: 1h 28m 18s
223:	test: 0.8113908	best: 0.8113908 (223)	t

314:	test: 0.8119341	best: 0.8119341 (314)	total: 36m 28s	remaining: 1h 19m 19s
315:	test: 0.8119704	best: 0.8119704 (315)	total: 36m 35s	remaining: 1h 19m 12s
316:	test: 0.8119736	best: 0.8119736 (316)	total: 36m 43s	remaining: 1h 19m 6s
317:	test: 0.8119723	best: 0.8119736 (316)	total: 36m 49s	remaining: 1h 18m 58s
318:	test: 0.8119710	best: 0.8119736 (316)	total: 36m 55s	remaining: 1h 18m 50s
319:	test: 0.8119753	best: 0.8119753 (319)	total: 37m 2s	remaining: 1h 18m 43s
320:	test: 0.8119813	best: 0.8119813 (320)	total: 37m 10s	remaining: 1h 18m 38s
321:	test: 0.8119816	best: 0.8119816 (321)	total: 37m 18s	remaining: 1h 18m 33s
322:	test: 0.8119816	best: 0.8119816 (321)	total: 37m 25s	remaining: 1h 18m 25s
323:	test: 0.8119811	best: 0.8119816 (321)	total: 37m 32s	remaining: 1h 18m 20s
324:	test: 0.8119821	best: 0.8119821 (324)	total: 37m 40s	remaining: 1h 18m 15s
325:	test: 0.8119808	best: 0.8119821 (324)	total: 37m 48s	remaining: 1h 18m 9s
326:	test: 0.8119842	best: 0.8119842 (326)	

418:	test: 0.8125012	best: 0.8125026 (416)	total: 49m 41s	remaining: 1h 8m 53s
419:	test: 0.8125019	best: 0.8125026 (416)	total: 49m 49s	remaining: 1h 8m 48s
420:	test: 0.8125014	best: 0.8125026 (416)	total: 49m 58s	remaining: 1h 8m 43s
421:	test: 0.8125009	best: 0.8125026 (416)	total: 50m 5s	remaining: 1h 8m 36s
422:	test: 0.8125013	best: 0.8125026 (416)	total: 50m 14s	remaining: 1h 8m 31s
423:	test: 0.8125300	best: 0.8125300 (423)	total: 50m 21s	remaining: 1h 8m 24s
424:	test: 0.8125331	best: 0.8125331 (424)	total: 50m 28s	remaining: 1h 8m 17s
425:	test: 0.8125339	best: 0.8125339 (425)	total: 50m 36s	remaining: 1h 8m 11s
426:	test: 0.8125358	best: 0.8125358 (426)	total: 50m 43s	remaining: 1h 8m 4s
427:	test: 0.8125445	best: 0.8125445 (427)	total: 50m 50s	remaining: 1h 7m 57s
428:	test: 0.8125454	best: 0.8125454 (428)	total: 50m 58s	remaining: 1h 7m 51s
429:	test: 0.8125460	best: 0.8125460 (429)	total: 51m 6s	remaining: 1h 7m 45s
430:	test: 0.8125464	best: 0.8125464 (430)	total: 51m 1

523:	test: 0.8129230	best: 0.8129232 (522)	total: 1h 2m 55s	remaining: 57m 9s
524:	test: 0.8129228	best: 0.8129232 (522)	total: 1h 3m 2s	remaining: 57m 1s
525:	test: 0.8129238	best: 0.8129238 (525)	total: 1h 3m 9s	remaining: 56m 54s
526:	test: 0.8129242	best: 0.8129242 (526)	total: 1h 3m 16s	remaining: 56m 47s
527:	test: 0.8129247	best: 0.8129247 (527)	total: 1h 3m 24s	remaining: 56m 40s
528:	test: 0.8129238	best: 0.8129247 (527)	total: 1h 3m 32s	remaining: 56m 34s
529:	test: 0.8129225	best: 0.8129247 (527)	total: 1h 3m 41s	remaining: 56m 29s
530:	test: 0.8129266	best: 0.8129266 (530)	total: 1h 3m 49s	remaining: 56m 22s
531:	test: 0.8129285	best: 0.8129285 (531)	total: 1h 3m 56s	remaining: 56m 15s
532:	test: 0.8129436	best: 0.8129436 (532)	total: 1h 4m 5s	remaining: 56m 9s
533:	test: 0.8129440	best: 0.8129440 (533)	total: 1h 4m 13s	remaining: 56m 2s
534:	test: 0.8129431	best: 0.8129440 (533)	total: 1h 4m 21s	remaining: 55m 56s
535:	test: 0.8129562	best: 0.8129562 (535)	total: 1h 4m 28s

627:	test: 0.8132843	best: 0.8132843 (627)	total: 1h 16m 28s	remaining: 45m 17s
628:	test: 0.8132921	best: 0.8132921 (628)	total: 1h 16m 36s	remaining: 45m 11s
629:	test: 0.8132914	best: 0.8132921 (628)	total: 1h 16m 44s	remaining: 45m 4s
630:	test: 0.8132915	best: 0.8132921 (628)	total: 1h 16m 53s	remaining: 44m 57s
631:	test: 0.8132916	best: 0.8132921 (628)	total: 1h 17m 1s	remaining: 44m 51s
632:	test: 0.8132939	best: 0.8132939 (632)	total: 1h 17m 10s	remaining: 44m 44s
633:	test: 0.8132935	best: 0.8132939 (632)	total: 1h 17m 17s	remaining: 44m 37s
634:	test: 0.8132959	best: 0.8132959 (634)	total: 1h 17m 25s	remaining: 44m 30s
635:	test: 0.8132966	best: 0.8132966 (635)	total: 1h 17m 33s	remaining: 44m 23s
636:	test: 0.8132964	best: 0.8132966 (635)	total: 1h 17m 41s	remaining: 44m 16s
637:	test: 0.8132940	best: 0.8132966 (635)	total: 1h 17m 49s	remaining: 44m 9s
638:	test: 0.8133343	best: 0.8133343 (638)	total: 1h 17m 58s	remaining: 44m 2s
639:	test: 0.8133333	best: 0.8133343 (638)	t

731:	test: 0.8137137	best: 0.8137151 (728)	total: 1h 28m 24s	remaining: 32m 22s
732:	test: 0.8137154	best: 0.8137154 (732)	total: 1h 28m 30s	remaining: 32m 14s
733:	test: 0.8137173	best: 0.8137173 (733)	total: 1h 28m 35s	remaining: 32m 6s
734:	test: 0.8137162	best: 0.8137173 (733)	total: 1h 28m 41s	remaining: 31m 58s
735:	test: 0.8137167	best: 0.8137173 (733)	total: 1h 28m 46s	remaining: 31m 50s
736:	test: 0.8137124	best: 0.8137173 (733)	total: 1h 28m 51s	remaining: 31m 42s
737:	test: 0.8137134	best: 0.8137173 (733)	total: 1h 28m 56s	remaining: 31m 34s
738:	test: 0.8137259	best: 0.8137259 (738)	total: 1h 29m 1s	remaining: 31m 26s
739:	test: 0.8137260	best: 0.8137260 (739)	total: 1h 29m 7s	remaining: 31m 18s
740:	test: 0.8137323	best: 0.8137323 (740)	total: 1h 29m 12s	remaining: 31m 10s
741:	test: 0.8137324	best: 0.8137324 (741)	total: 1h 29m 18s	remaining: 31m 3s
742:	test: 0.8137317	best: 0.8137324 (741)	total: 1h 29m 23s	remaining: 30m 55s
743:	test: 0.8137409	best: 0.8137409 (743)	t

834:	test: 0.8140182	best: 0.8140214 (822)	total: 1h 38m 2s	remaining: 19m 22s
835:	test: 0.8140215	best: 0.8140215 (835)	total: 1h 38m 8s	remaining: 19m 15s
836:	test: 0.8140225	best: 0.8140225 (836)	total: 1h 38m 12s	remaining: 19m 7s
837:	test: 0.8140223	best: 0.8140225 (836)	total: 1h 38m 18s	remaining: 19m
838:	test: 0.8140296	best: 0.8140296 (838)	total: 1h 38m 23s	remaining: 18m 52s
839:	test: 0.8140301	best: 0.8140301 (839)	total: 1h 38m 29s	remaining: 18m 45s
840:	test: 0.8140303	best: 0.8140303 (840)	total: 1h 38m 34s	remaining: 18m 38s
841:	test: 0.8140357	best: 0.8140357 (841)	total: 1h 38m 39s	remaining: 18m 30s
842:	test: 0.8140377	best: 0.8140377 (842)	total: 1h 38m 45s	remaining: 18m 23s
843:	test: 0.8140374	best: 0.8140377 (842)	total: 1h 38m 50s	remaining: 18m 16s
844:	test: 0.8140398	best: 0.8140398 (844)	total: 1h 38m 56s	remaining: 18m 8s
845:	test: 0.8140442	best: 0.8140442 (845)	total: 1h 39m 2s	remaining: 18m 1s
846:	test: 0.8140446	best: 0.8140446 (846)	total: 

938:	test: 0.8142123	best: 0.8142130 (937)	total: 1h 47m 37s	remaining: 6m 59s
939:	test: 0.8142152	best: 0.8142152 (939)	total: 1h 47m 42s	remaining: 6m 52s
940:	test: 0.8142140	best: 0.8142152 (939)	total: 1h 47m 49s	remaining: 6m 45s
941:	test: 0.8142145	best: 0.8142152 (939)	total: 1h 48m 1s	remaining: 6m 39s
942:	test: 0.8142147	best: 0.8142152 (939)	total: 1h 48m 9s	remaining: 6m 32s
943:	test: 0.8142142	best: 0.8142152 (939)	total: 1h 48m 17s	remaining: 6m 25s
944:	test: 0.8142142	best: 0.8142152 (939)	total: 1h 48m 25s	remaining: 6m 18s
945:	test: 0.8142138	best: 0.8142152 (939)	total: 1h 48m 32s	remaining: 6m 11s
946:	test: 0.8142139	best: 0.8142152 (939)	total: 1h 48m 40s	remaining: 6m 4s
947:	test: 0.8142130	best: 0.8142152 (939)	total: 1h 48m 47s	remaining: 5m 58s
948:	test: 0.8142115	best: 0.8142152 (939)	total: 1h 48m 55s	remaining: 5m 51s
949:	test: 0.8142119	best: 0.8142152 (939)	total: 1h 49m 4s	remaining: 5m 44s
950:	test: 0.8142111	best: 0.8142152 (939)	total: 1h 49m

In [56]:
print('the validation accuracy is :{:.6f}'.format(accuracy_score(y_valid,model.predict(X_valid))))
print(roc_auc_score(y_valid,model.predict(X_valid)))

the validation accuracy is :0.968437
0.5162622391117986


In [59]:
submission_df['id'] = test_df['id']
submission_df['is_chat'] = pd.DataFrame(model.predict_proba(s2.drop(['id'],axis=1))[:,1])
submission_df.to_csv('submission_df_catboost_baseline.csv',index= False)

In [71]:
r1 = 0.4*submission_df['is_chat']
r2 = pd.read_csv('submission_df_lgbm_baseline.csv')
r3= 0.6*r2['is_chat']
r4['id'] = test_df['id']
r4['is_chat'] = pd.DataFrame(r1+r3)

In [73]:
cols = r4.columns.tolist()

In [74]:
cols

['is_chat', 'id']

In [75]:
cols = cols[-1:] + cols[:-1]

In [76]:
cols

['id', 'is_chat']

In [77]:
r4 = r4[cols]
r4.columns

Index(['id', 'is_chat'], dtype='object')

In [78]:
r4.to_csv('ensemble_lightgbm_catboost.csv',index= False)

In [41]:
r0 = pd.read_csv('submission_df_lg_baseline_v2.csv')
r1 = 0.4*r0['is_chat']
r2 = pd.read_csv('submission_df_lgbm_baseline.csv')
r3= 0.6*r2['is_chat']
r4 = pd.DataFrame()
r4['id'] = test_df['id']
r4['is_chat'] = pd.DataFrame(r1+r3)

In [42]:
r4.columns

Index(['id', 'is_chat'], dtype='object')

In [43]:
r4.to_csv('ensemble_lightgbm_logistic.csv',index= False)